In [1]:
pip install pyclustertend

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyclustertend in c:\users\user\anaconda3\lib\site-packages (1.4.9)



You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install bioinfokit

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import math
import datetime as dt
import re
from os.path import join
import os
from sklearn.base import clone

from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sompy.visualization.mapview import View2D
from sompy.visualization.bmuhits import BmuHitsView
from sompy.visualization.hitmap import HitMapView

from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score, silhouette_samples

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer
from pyclustertend import hopkins
from pyclustertend import vat
from pyclustertend import ivat
from sklearn.utils import shuffle

from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
import sompy

%matplotlib inline
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'
sns.set_style("whitegrid")

Loaded backend module://ipykernel.pylab.backend_inline version unknown.


# Import PVA Database

In [4]:
pva = pd.read_csv('donors.csv', low_memory = False, index_col = 'CONTROLN')
pva.drop(columns = 'Unnamed: 0', inplace = True)
pva.sort_index(inplace = True)

FileNotFoundError: [Errno 2] No such file or directory: 'donors.csv'

# Feature Engineering

In [ ]:
#give meaninful names to Gender values and group missing values as unknown 
pva['GENDER'].replace({' ':'U', 'C':'U', 'A':'U'}, regex = True, inplace = True)
pva['GENDER'].replace({'F':'Female', 'M':'Male', 'J':'Join', 'U':'Unknown'}, regex = True, inplace = True)

In [ ]:
#extract Urbancy level from Domain feature 
pva['URBANICITY_LEVEL'] = pva['DOMAIN'].str.split('([0-9])', 1, expand = True).iloc[:,0]

In [ ]:
#give meaninful names to Urbanicity Level values and group missing values as nan
pva['URBANICITY_LEVEL'].replace({' ': np.nan,'U':'Urban', 'C':'City', 'S':'Suburban', 'T':'Town', 'R':'Rural'}, inplace = True)

In [ ]:
#rename RFA_2F in a meaninful name
pva.rename(columns={'RFA_2F': 'GIFT_FREQUENCY'}, inplace=True)

In [ ]:
#create a function that convert all dates to datetime format
def date_time(data, column):
    '''Transform to date time'''
    data[column] = pd.to_datetime(data[column])

#create a list containing all the date features
date_features = ['ODATEDW', 'DOB', 'ADATE_2', 'ADATE_3', 'ADATE_4', 'ADATE_5', 'ADATE_6', 'ADATE_7', 'ADATE_8', 'ADATE_9', 'ADATE_10', 'ADATE_11', 'ADATE_12', 'ADATE_13', 'ADATE_14', 
         'ADATE_15', 'ADATE_16', 'ADATE_17', 'ADATE_18', 'ADATE_19', 'ADATE_20', 'ADATE_21', 'ADATE_22', 'ADATE_23', 'ADATE_24', 'MAXADATE', 'RDATE_3', 'RDATE_4', 'RDATE_5',
         'RDATE_6', 'RDATE_7', 'RDATE_8', 'RDATE_9', 'RDATE_10', 'RDATE_11', 'RDATE_12', 'RDATE_13', 'RDATE_14',  'RDATE_15', 'RDATE_16', 'RDATE_17', 'RDATE_18', 'RDATE_19',
         'RDATE_20', 'RDATE_21', 'RDATE_22', 'RDATE_23', 'RDATE_24', 'MINRDATE', 'MAXRDATE', 'LASTDATE', 'FISTDATE', 'NEXTDATE']

#apply the function
date_features_list = []
for date in date_features:
    if date in pva.columns:
        date_time(pva, date)
        date_features_list.append(date)

In [ ]:
#convert DOB to Age feature and drop DOB from dataset
pva['AGE'] = (pd.Timestamp('today') - pva['DOB']).astype('<m8[Y]')
pva.drop('DOB', axis =1, inplace = True)

In [ ]:
#convert ODATEDW (date of donors first gift) to number of years since first gift 
pva['YEARS_FIRST_GIFT'] = (pd.Timestamp('today') - pva['ODATEDW']).astype('<m8[Y]')
pva.drop('ODATEDW', axis =1, inplace = True)

In [ ]:
#replace missing values of interest features with No
donor_interests = ['MBCRAFT', 'MBGARDEN', 'MBBOOKS', 'MBCOLECT', 'MAGFAML', 'MAGFEM', 'MAGMALE','PUBGARDN',
                   'PUBCULIN','PUBHLTH','PUBDOITY', 'PUBNEWFN', 'PUBPHOTO','PUBOPP','COLLECT1','VETERANS',
                   'BIBLE','CATLG','HOMEE','PETS','CDPLAY','STEREO','PCOWNERS', 'PHOTO','CRAFTS','FISHER',
                   'GARDENIN','BOATS','WALKER','KIDSTUFF','CARDS','PLATES']

for interest in donor_interests:
    if interest in pva.columns:
        pva[interest] = pva[interest].replace(' ', 'N')

In [ ]:
#create log variable for continous variables that doesn't have 0
pva['LOG_RAMNTALL'] = np.log(pva['RAMNTALL']) 
pva['LOG_AGE'] = np.log(pva['AGE']) 
pva['LOG_AVGGIFT'] = np.log(pva['AVGGIFT'])

# Feature Selection

In [ ]:
#create a daba base with only the metric features selected
pva_metric = pva[['INCOME', 'LASTGIFT', 'MINRAMNT', 'LOG_RAMNTALL','LOG_AGE', 'LOG_AVGGIFT', 
             'YEARS_FIRST_GIFT', 'NGIFTALL', 'GIFT_FREQUENCY', 'TIMELAG', 'HIT', 'CARDGIFT']]

#create a daba base with only the categorical features selected
pva_non_metric = pva[['STATE', 'GENDER', 'URBANICITY_LEVEL','COLLECT1','VETERANS','BIBLE','CATLG','HOMEE','PETS',
                       'CDPLAY','STEREO','PCOWNERS', 'PHOTO','CRAFTS','FISHER','GARDENIN','BOATS','WALKER',
                       'KIDSTUFF','CARDS','PLATES']]

In [ ]:
#create a list with the names of the metric features
pva_metric_list = pva_metric.columns.tolist()

#create a list with the names of the categorical features
pva_non_metric_list = pva_non_metric.columns.tolist()

# Data Exploration

## Metric Features

In [ ]:
pva_metric.head()

In [ ]:
pva_metric.info()

In [ ]:
pva_metric.info()

In [ ]:
pva_metric.describe().T

## Non-Metric Features

In [ ]:
pva_non_metric.head()

In [ ]:
pva_non_metric.info()

In [ ]:
pva_non_metric.describe().T

## Visualization 

In [ ]:
#Create a grid to visualize Age, Years_first_gift, Gift_frecuency, Cardgift, Income, Gender and Urbanicity level
f = plt.figure(figsize = (30,20))
gs = f.add_gridspec(4, 2)

with sns.axes_style("whitegrid"):
    
    ax = f.add_subplot(gs[0, 0])
    sns.histplot(x = 'AGE', data = pva, bins = 10, ax = ax, kde = True, hue = 'INCOME', palette = 'viridis')

    ax = f.add_subplot(gs[0, 1])
    sns.countplot(x = 'YEARS_FIRST_GIFT', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[1, 0])
    sns.countplot(x = 'GIFT_FREQUENCY', data = pva, ax = ax, palette = 'viridis')

    ax = f.add_subplot(gs[1, 1])
    sns.countplot(x = 'CARDGIFT', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[2, 0])
    sns.countplot(x = 'INCOME', data = pva, ax = ax, palette = 'viridis')  
    
    ax = f.add_subplot(gs[2, 1])
    sns.countplot(x = 'GENDER', data = pva, ax = ax, palette = 'viridis') 
    
    ax = f.add_subplot(gs[3, 0])
    sns.countplot(x = 'URBANICITY_LEVEL', data = pva, ax = ax, palette = 'viridis') 


In [ ]:
#create a grid to plot Minramnt and Lastgift
f = plt.figure(figsize = (20,10))
gs = f.add_gridspec(2, 1)

with sns.axes_style("whitegrid"):
    
    ax = f.add_subplot(gs[0, 0])
    sns.histplot(x = 'MINRAMNT', bins = 90, data = pva, ax = ax, palette = 'viridis')
    plt.xlim([0, 60])
        
    ax = f.add_subplot(gs[1, 0])
    sns.histplot(x = 'LASTGIFT', bins = 90, data = pva, ax = ax, palette = 'viridis') 
    plt.xlim([0, 60])

In [ ]:
#visualize log_ramntall
a4_dims = (20, 10)
fig, ax = pyplot.subplots(figsize = a4_dims)

sns.histplot(x = np.log(pva['RAMNTALL']), bins = 30,  ax = ax, palette = 'viridis', kde=True) ## data = pva,
plt.xlim([0, 10])

In [ ]:
#visualize Log_avggift
a4_dims = (20, 10)
fig, ax = pyplot.subplots(figsize = a4_dims)

sns.histplot(x = np.log(pva['AVGGIFT']), bins = 30,  ax = ax, palette = 'viridis', kde=True) ## data = pva,
plt.xlim([0, 10])

In [ ]:
#visualize Log_age
a4_dims = (20, 10)
fig, ax = pyplot.subplots(figsize = a4_dims)

sns.histplot(x = np.log(pva['AGE']),  ax = ax, palette = 'viridis', kde=True) ## data = pva,
plt.xlim([0, 10])

In [ ]:
#visualize Nigiftall
sns.displot(x = 'NGIFTALL', data = pva, binwidth= 10, palette='viridis')
plt.xlim([0, 50])

In [ ]:
#visualize Hit
sns.displot(x = 'HIT', data = pva, palette = 'viridis', binwidth= 10)
plt.xlim([0, 40])

In [ ]:
#visualice donnor by state ordered in descending manner
a4_dims = (20, 10)
fig, ax = pyplot.subplots(figsize = a4_dims)
sns.countplot(x = 'STATE', data = pva, ax = ax, palette = 'viridis', 
                 order = pva['STATE'].value_counts().index)

In [ ]:
#visualize nterests
f = plt.figure(figsize = (20,30))
gs = f.add_gridspec(6, 3)

with sns.axes_style("whitegrid"):
    
    ax = f.add_subplot(gs[0, 0])
    sns.countplot(x = 'PLATES', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[0, 1])
    sns.countplot(x = 'COLLECT1', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[0, 2])
    sns.countplot(x = 'VETERANS', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[1, 0])
    sns.countplot(x = 'BIBLE', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[1, 1])
    sns.countplot(x = 'CATLG', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[1, 2])
    sns.countplot(x = 'HOMEE', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[2, 0])
    sns.countplot(x = 'PETS', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[2, 1])
    sns.countplot(x = 'CDPLAY', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[2, 2])
    sns.countplot(x = 'STEREO', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[3, 0])
    sns.countplot(x = 'PCOWNERS', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[3, 1])
    sns.countplot(x = 'PHOTO', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[3, 2])
    sns.countplot(x = 'CRAFTS', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[4, 0])
    sns.countplot(x = 'FISHER', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[4, 1])
    sns.countplot(x = 'GARDENIN', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[4, 2])
    sns.countplot(x = 'BOATS', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[5, 0])
    sns.countplot(x = 'WALKER', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[5, 1])
    sns.countplot(x = 'KIDSTUFF', data = pva, ax = ax, palette = 'viridis')
    
    ax = f.add_subplot(gs[5, 2])
    sns.countplot(x = 'CARDS', data = pva, ax = ax, palette = 'viridis')
    
    

# Filling Missing Values

In [ ]:
# Use KNNImputer to fill missing values on numeric features
imputer = KNNImputer(n_neighbors=5, weights = 'uniform')
pva_metric[['LOG_AGE', 'INCOME']] = imputer.fit_transform(pva_metric[['LOG_AGE', 'INCOME']])

In [ ]:
#create a list with categorical features with missing values
mode_as_missing_value = ['URBANICITY_LEVEL']

In [ ]:
#fill missing values in non metric features with mode
for missing_feature in mode_as_missing_value:
    pva_non_metric[missing_feature].fillna(pva_non_metric[missing_feature].mode()[0], inplace = True)

In [ ]:
#fill nan values of Timelag with 0
pva_metric['TIMELAG'].fillna(0, inplace=True)

In [ ]:
#check is missing values of categorical features were correctly filled
pva_non_metric.isna().any()

In [ ]:
#check is missing values of metric features were correctly filled
pva_metric.isna().any()

# Outlier Removal

In [ ]:
#create an IQR with the 10th and 90th quantiles
q10 = pva_metric.quantile(.10)
q90 = pva_metric.quantile(.90)
iqr = (q90 - q10)

upper_lim = q90 + 1.5*iqr
lower_lim = q10 - 1.5*iqr

In [ ]:
#create a filter with the observations out of the IQR
filters = []
for metric in pva_metric_list:
    llim = lower_lim[metric]
    ulim = upper_lim[metric]
    filters.append(pva_metric[metric].between(llim, ulim, inclusive=True))

filters = pd.Series(np.all(filters, 0), index = pva_metric.index.tolist())   

#print the percentage of data kept after applying the filter created above
print('Percentage of data kept after removing outliers:', np.round(pva_metric[filters].shape[0]/pva_metric.shape[0], 2))

#replace the metric data base for the filtered one
pva_metric = pva_metric[filters]

In [ ]:
#replace the categorical data base for the filtered one
pva_non_metric = pva_non_metric[filters]

# Data Normalization

In [ ]:
#create a copy of data base with missing values filled and outlier removed
pva_standard = pva_metric.copy()

In [ ]:
#scaling data using standarscaler to get a mean of 0 and variance of 1
scaler = StandardScaler()
scaled_feat = scaler.fit_transform(pva_standard)

In [ ]:
#create a data base with standarized metric features
pva_standard[pva_metric_list] = scaled_feat

In [ ]:
pva_standard

# Correlation Matrix

In [ ]:
fig = plt.figure(figsize=(10, 8))

#obtain correlation matrix. Round the values to 2 decimal cases.
corr_matrix = np.round(pva_metric[pva_metric_list].corr(method='pearson'), decimals=2)

#build annotation matrix (values above |0.5| will appear annotated in the plot)
mask_annot = np.absolute(corr_matrix.values) >= 0.5
annot = np.where(mask_annot, corr_matrix.values, np.full(corr_matrix.shape,''))

#plot heatmap of the correlation matrix
sns.heatmap(data=corr_matrix, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True), 
            fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

fig.subplots_adjust(top=0.95)
fig.suptitle("Correlation Matrix", fontsize=20)

plt.show()

# Assessing Clustering Tendency

## Visual Assesment Tendency

In [ ]:
#apply vat algorithm to dataset 
vat(pva_standard)

## Hopkins statistic

In [ ]:
#apply Hopkins Statistic to data set
hopkins(pva_standard, len(pva_standard))

# One Hot Encoding

In [ ]:
#create a copy to encode categorial features
pva_non_metric_ohc = pva_non_metric.copy()

In [ ]:
#apply the method OneHotEncoding
ohc = OneHotEncoder(sparse=False, drop= 'if_binary')
ohc_feat = ohc.fit_transform(pva_non_metric_ohc)
ohc_feat_names = ohc.get_feature_names(pva_non_metric_list)
ohc_pva = pd.DataFrame(ohc_feat, index = pva_non_metric_ohc.index, columns = ohc_feat_names)  
ohc_pva

# Dimensionality Reduction

In [ ]:
#create a copy to do PCA
pva_pca = pva_standard.copy()

In [ ]:
#apply PCA to all metric features 
pca = PCA(n_components = pva_pca.shape[1], whiten = True)
pca_reduced = pca.fit_transform(pva_pca)

In [ ]:
#The following table show the cumulative variance explained by the PC
pd.DataFrame(
    {"Eigenvalue": pca.explained_variance_,
     "Difference": np.insert(np.diff(pca.explained_variance_), 0, 0),
     "Proportion": pca.explained_variance_ratio_,
     "Cumulative": np.cumsum(pca.explained_variance_ratio_)},
    index=range(1, pca.n_components_ + 1)
)

In [ ]:
#check the number of components have a acumulated variance bigger that 90%
np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.9)

In [ ]:

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# draw plots
ax1.plot(pca.explained_variance_, marker=".", markersize=12)
ax2.plot(pca.explained_variance_ratio_, marker=".", markersize=12, label="Proportion")
ax2.plot(np.cumsum(pca.explained_variance_ratio_), marker=".", markersize=12, linestyle="--", label="Cumulative")

# customizations
ax2.legend()
ax1.set_title("Scree Plot", fontsize=14)
ax2.set_title("Variance Explained", fontsize=14)
ax1.set_ylabel("Eigenvalue")
ax2.set_ylabel("Proportion")
ax1.set_xlabel("Components")
ax2.set_xlabel("Components")
ax1.set_xticks(range(0, pca.n_components_, 2))
ax1.set_xticklabels(range(1, pca.n_components_ + 1, 2))
ax2.set_xticks(range(0, pca.n_components_, 2))
ax2.set_xticklabels(range(1, pca.n_components_ + 1, 2))
plt.xticks(rotation = 90)

plt.show()

In [ ]:
#create a list with the PC that explain 90% of the variance
pca_feat_names = [f"PC{i}" for i in range(np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.9))]

In [ ]:
#create a dataframe with the PC that explain 90% of the variance
pca_90_pct = pd.DataFrame(pca_reduced[:,0:6], index= pva_pca.index, columns = pca_feat_names) 

In [ ]:
#plot data with the PC
plt.scatter(pca_90_pct['PC0'], pca_90_pct['PC1'])

# K-Means Clustering

## Define Number of Clusters

### Elbow Method

In [ ]:
range_clusters = range(2,11)

In [ ]:
#iterate over a desired ncluster range and save the inertia of the given cluster solution
inertia = []
for k in range_clusters: 
    kmclust = KMeans(n_clusters = k, init = 'k-means++', n_init = 15, random_state = 42)
    kmclust.fit(pva_standard)
    inertia.append(kmclust.inertia_)  

In [ ]:
#plot inertia vs number of clusters
plt.figure(figsize=(9,5))
plt.plot(range_clusters , inertia) # range_clusters ,
plt.ylabel("Inertia: SSw")
plt.xlabel("Number of clusters")
plt.title("Inertia plot over clusters", size=15)
plt.show()

### Calinski-Harabasz Index

In [ ]:
Calinski_Harabasz = []
for k in range_clusters:
    kmeans_model = KMeans(n_clusters = k, init = 'k-means++', n_init = 15,random_state = 42).fit(pva_standard)
    labels = kmeans_model.labels_
    labels = kmeans_model.labels_ 
    Calinski_Harabasz.append(metrics.calinski_harabasz_score(pva_standard, labels))

In [ ]:
# The Calinski_Harabasz plot
plt.figure(figsize=(9,5))
plt.plot(range_clusters , Calinski_Harabasz) 
plt.ylabel("Calinski Harabasz")
plt.xlabel("Number of clusters")
plt.title("Calinski Harabasz plot over clusters", size=15)
plt.show()

## K-Means Clustering Solution

In [ ]:
km_clust = KMeans(n_clusters = 4, init = 'k-means++', n_init = 15, random_state = 42)
km_labels = km_clust.fit_predict(pva_standard)

## Characterizing K-Means

### Metric Features

In [ ]:
pva_standard_km_labeled = pd.concat((pva_standard, pd.Series(km_labels, name='labels', index=pva_standard.index.tolist())), axis=1)
km_standard = pva_standard_km_labeled.groupby('labels').mean()
km_standard['label'] = km_standard.index
km_standard

In [ ]:
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(km_standard, class_column = 'label', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=90)

In [ ]:
pva_metric_km_labeled = pd.concat((pva_metric, pd.Series(km_labels, name='labels', index=pva_standard.index.tolist())), axis=1)
km_metric = pva_metric_km_labeled.groupby('labels').mean()
km_metric['label'] = km_metric.index
km_metric 

In [ ]:
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(km_metric, class_column = 'label', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=90)

### Categorical Features

In [ ]:
pva_non_metric_km_labeled = pd.concat((ohc_pva, pd.Series(km_labels, name='labels', index=pva_standard.index.tolist())), axis=1)

#### State

In [ ]:
km_state = pva_non_metric_km_labeled.groupby('labels').mean().iloc[:,:57]
km_state['label'] = km_state.index
km_state 

In [ ]:
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(km_state, class_column = 'label', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=90)

#### Gender

In [ ]:
km_gender = pva_non_metric_km_labeled.groupby('labels').mean().iloc[:,57:61]
km_gender['label'] = km_gender.index
km_gender

In [ ]:
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(km_gender, class_column = 'label', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=40)

#### Urbanicity Level

In [ ]:
km_urbancity = pva_non_metric_km_labeled.groupby('labels').mean().iloc[:,61:66]
km_urbancity['label'] = km_urbancity.index
km_urbancity

In [ ]:
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(km_urbancity, class_column = 'label', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=40)

#### Interests

In [ ]:
km_interests = pva_non_metric_km_labeled.groupby('labels').mean().iloc[:,66:]
km_interests['label'] = km_interests.index
km_interests

In [ ]:
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(km_interests, class_column = 'label', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=40)

## Visualization K-means

In [ ]:
two_dim_km = TSNE(random_state=42).fit_transform(pva_standard_km_labeled)

In [ ]:
pd.DataFrame(two_dim_km).plot.scatter(x=0, y=1, c = pva_standard_km_labeled['labels'], colormap='viridis', figsize=(15,10))
plt.show()

# Hierarchical and K-means Clustering

### Define large number of K-means clusters

In [ ]:
km = KMeans(n_clusters = 30, init = 'k-means++', n_init = 10, random_state = 0)
km.fit(pva_standard)

In [ ]:
km_labels_2 = km.predict(pva_standard)

In [ ]:
km_labeled_2 = pd.concat((pva_standard, pd.Series(km_labels_2, name='Labels_km', index=pva_standard.index.tolist())), axis=1)
km_centroids_2 = km_labeled_2.groupby('Labels_km').mean()

In [ ]:
km_labeled_metric_2 = pd.concat((pva_metric, pd.Series(km_labels_2, name='Labels_km', index=pva_metric.index.tolist())), axis=1)
km_centroids_metric_2 = km_labeled_metric_2.groupby('Labels_km').mean()

In [ ]:
km_labeled_non_metric_2 = pd.concat((ohc_pva, pd.Series(km_labels_2, name='Labels_km', index=pva_standard.index.tolist())), axis=1)
km_centriods_non_metric_2  = km_labeled_non_metric_2.groupby('Labels_km').mean()

### Defining number of hierarquical clusters:

In [ ]:
# setting distance_threshold=0 and n_clusters=None ensures we compute the full tree
linkage = 'ward'
distance = 'euclidean'
hclust_1 = AgglomerativeClustering(linkage=linkage, affinity=distance, n_clusters=None, distance_threshold=0.000001)
hclust_1.fit_predict(km_centroids_2)

In [ ]:
# Adapted from:
# create the counts of samples under each node (number of points being merged)
counts = np.zeros(hclust_1.children_.shape[0])
n_samples = len(hclust_1.labels_)

# hclust.children_ contains the observation ids that are being merged together
# At the i-th iteration, children[i][0] and children[i][1] are merged to form node n_samples + i
for i, merge in enumerate(hclust_1.children_):
    # track the number of observations in the current cluster being formed
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            # If this is True, then we are merging an observation
            current_count += 1  # leaf node
        else:
            # Otherwise, we are merging a previously formed cluster
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count

# the hclust.children_ is used to indicate the two points/clusters being merged (dendrogram's u-joins)
# the hclust.distances_ indicates the distance between the two points/clusters (height of the u-joins)
# the counts indicate the number of points being merged (dendrogram's x-axis)
linkage_matrix = np.column_stack(
    [hclust_1.children_, hclust_1.distances_, counts]
).astype(float)

# Plot the corresponding dendrogram
sns.set()
fig = plt.figure(figsize=(11,5))
# The Dendrogram parameters need to be tuned
y_threshold = 6
dendrogram(linkage_matrix, truncate_mode='level', p=5, color_threshold=y_threshold, above_threshold_color='k')
plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
plt.title(f'Hierarchical Clustering - {linkage.title()}\'s Dendrogram', fontsize=21)
plt.xlabel('Number of points in node (or index of point if no parenthesis)')
plt.ylabel(f'{distance.title()} Distance', fontsize=13)
plt.show()

## Hierarchical on top of K-means

In [ ]:
# # Performing Hierarchical Clustering
hc_km_cluster = AgglomerativeClustering(linkage='ward', affinity = 'euclidean', n_clusters = 5)
hc_km_labels = hc_km_cluster.fit_predict(km_centroids_2)

## Characterizing Hierarchical on top of K-means

### Metric Features

In [ ]:
hc_km_labeled_standard = pd.concat((km_centroids_2, pd.Series(hc_km_labels, name='Labels_hc')), axis=1)
hc_km_standard = hc_km_labeled_standard.groupby('Labels_hc').mean()
hc_km_standard['Label_hc'] = hc_km_standard.index
hc_km_standard 

In [ ]:
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(hc_km_standard, class_column = 'Label_hc', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=90)

In [ ]:
hc_km_labeled_metric = pd.concat((km_centroids_metric_2, pd.Series(hc_km_labels, name='Labels_hc')), axis=1)
hc_km_metric = hc_km_labeled_metric.groupby('Labels_hc').mean()
hc_km_metric['Label_hc'] = hc_km_metric.index
hc_km_metric

In [ ]:
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(hc_km_metric, class_column = 'Label_hc', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=90)

### Categorical Features

In [ ]:
hc_km_labeled_non_metric = pd.concat((km_centriods_non_metric_2, pd.Series(hc_km_labels, name='Labels_hc')), axis=1)

#### State

In [ ]:
hc_km_state = hc_km_labeled_non_metric.groupby('Labels_hc', as_index=False).mean().iloc[:,1:57]
hc_km_state['label'] = hc_km_state.index
hc_km_state

In [ ]:
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(hc_km_state, class_column = 'label', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=90)

#### Gender

In [ ]:
hc_km_gender = hc_km_labeled_non_metric.groupby('Labels_hc', as_index=False).mean().iloc[:,57:61]
hc_km_gender['label'] = hc_km_gender.index
hc_km_gender

In [ ]:
#plt.plot(hc_km_labeled_non_metric.iloc[:,61:66])
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(hc_km_gender, class_column = 'label', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=40)

#### Urbanicity Level

In [ ]:
hc_km_urbanicity = hc_km_labeled_non_metric.groupby('Labels_hc', as_index=False).mean().iloc[:,61:66]
hc_km_urbanicity['label'] = hc_km_urbanicity.index
hc_km_urbanicity

In [ ]:
#plt.plot(hc_km_labeled_non_metric.iloc[:,61:66])
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(hc_km_urbanicity, class_column = 'label', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=40)

#### Interests

In [ ]:
hc_km_interests = hc_km_labeled_non_metric.groupby('Labels_hc', as_index=False).mean().iloc[:,66:]
hc_km_interests['label'] = hc_km_interests.index
hc_km_interests

In [ ]:
plt.figure(figsize=(15, 7))
pd.plotting.parallel_coordinates(hc_km_interests, class_column = 'label', color=sns.color_palette())
plt.xticks(fontsize=10, rotation=40)

## Visualization K-means on top of Hierarchical

In [ ]:
km_hc_tsne = km_labeled_2.copy()

In [ ]:
km_hc_tsne['Labels_km'].replace({0: 4, 1: 2, 2: 0, 3: 1, 4: 2, 5: 2, 6: 0, 7: 0, 8: 0, 9: 2, 10: 0, 11: 3, 12: 2, 13: 1, 14: 0, 15: 0, 16: 3, 17: 0, 18: 1, 19: 0, 20: 3, 21: 1, 22: 0, 23: 3, 24: 4, 25: 2, 26: 2, 27: 0, 28: 1, 29: 4}, inplace=True)

In [ ]:
km_hc_tsne.rename(columns={'Labels_km':'Labels_km_hc'}, inplace=True)

In [ ]:
two_dim_km_hc = TSNE(random_state=42).fit_transform(km_hc_tsne)

In [ ]:
pd.DataFrame(two_dim_km_hc).plot.scatter(x=0, y=1, c = km_hc_tsne['Labels_km_hc'], colormap='viridis', figsize=(15,10))
plt.show()

# Clustering by Perspectives

## K-means Clustering by Perspective

In [ ]:
Demographic = pva_standard[['INCOME', 'LOG_AGE', 'LOG_AVGGIFT', 'LASTGIFT', 'MINRAMNT', 'LOG_RAMNTALL']]
Engagement =  pva_standard[['YEARS_FIRST_GIFT', 'NGIFTALL', 'GIFT_FREQUENCY', 'TIMELAG', 'HIT', 'CARDGIFT']]

In [ ]:
Demographic_metric = pva_metric[['INCOME', 'LOG_AGE', 'LOG_AVGGIFT', 'LASTGIFT', 'MINRAMNT', 'LOG_RAMNTALL']]
Engagement_metric =  pva_metric[['YEARS_FIRST_GIFT', 'NGIFTALL', 'GIFT_FREQUENCY', 'TIMELAG', 'HIT', 'CARDGIFT']]

In [ ]:
def get_ss(df):
    """Computes the sum of squares for all variables given a dataset
    """
    ss = np.sum(df.var() * (df.count() - 1))
    return ss  # return sum of sum of squares of each df variable

def r2(df, labels):
    sst = get_ss(df)
    ssw = np.sum(df.groupby(labels).apply(get_ss))
    return 1 - ssw/sst
    
def get_r2_scores(df, clusterer, min_k=2, max_k=10):
    """
    Loop over different values of k. To be used with sklearn clusterers.
    """
    r2_clust = {}
    for n in range(min_k, max_k):
        clust = clone(clusterer).set_params(n_clusters=n)
        labels = clust.fit_predict(df)
        r2_clust[n] = r2(df, labels)
    return r2_clust

In [ ]:
# Set up the clusterers
kmeans = KMeans(init='k-means++', n_init=15, random_state=42)


# Obtaining the R² scores for each cluster solution on product variables
r2_scores = {}
r2_scores['kmeans_Demo'] = get_r2_scores(Demographic, kmeans)
r2_scores['kmeans_Eng'] = get_r2_scores(Engagement, kmeans)


# Visualizing the R² scores for each cluster solution on product variables
pd.DataFrame(r2_scores).plot.line(figsize=(10,7))

 
plt.title("Product Variables:\nR2 plot for various clustering methods\n", fontsize=21)
plt.legend(title = 'Cluster methods', title_fontsize=11)
plt.xlabel('Number of clusters', fontsize=13)
plt.ylabel('R2 metric', fontsize=13)
plt.show()

In [ ]:
kmeans_Demo = KMeans(n_clusters = 6, init = 'k-means++', n_init = 15, random_state=42)
Demo_labels = kmeans_Demo.fit_predict(Demographic)

kmeans_Eng = KMeans(n_clusters = 7, init = 'k-means++', n_init = 15, random_state = 42)
Eng_labels = kmeans_Eng.fit_predict(Engagement)

km_metric_perspective = pva_standard.copy()

km_metric_perspective['Demo_labels'] = Demo_labels
km_metric_perspective['Eng_labels'] = Eng_labels

In [ ]:
km_metric_perspective.groupby(['Demo_labels', 'Eng_labels'])\
    .size()\
    .to_frame()\
    .reset_index()\
    .pivot('Demo_labels', 'Eng_labels', 0)

## Merging Using Hierarchical Clustering

In [ ]:
perspective_centroids = km_metric_perspective.groupby(['Demo_labels', 'Eng_labels']).mean()

In [ ]:
linkage = 'ward'
hclust = AgglomerativeClustering(linkage = linkage, affinity='euclidean', distance_threshold = 0, n_clusters= None) # distance_threshold = 0
hclust_labels = hclust.fit_predict(perspective_centroids)

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py

# create the counts of samples under each node (number of points being merged)
counts = np.zeros(hclust.children_.shape[0])
n_samples = len(hclust.labels_)

# hclust.children_ contains the observation ids that are being merged together
# At the i-th iteration, children[i][0] and children[i][1] are merged to form node n_samples + i
for i, merge in enumerate(hclust.children_):
    # track the number of observations in the current cluster being formed
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            # If this is True, then we are merging an observation
            current_count += 1  # leaf node
        else:
            # Otherwise, we are merging a previously formed cluster
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count

# the hclust.children_ is used to indicate the two points/clusters being merged (dendrogram's u-joins)
# the hclust.distances_ indicates the distance between the two points/clusters (height of the u-joins)
# the counts indicate the number of points being merged (dendrogram's x-axis)
linkage_matrix = np.column_stack(
    [hclust.children_, hclust.distances_, counts]
).astype(float)

 

# Plot the corresponding dendrogram
sns.set()
fig = plt.figure(figsize=(11,5))
# The Dendrogram parameters need to be tuned
y_threshold=6
dendrogram(linkage_matrix, truncate_mode='level', labels = perspective_centroids.index, p=5, above_threshold_color='k')
plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
plt.title(f'Hierarchical Clustering - {linkage.title()}\'s Dendrogram', fontsize=21)
plt.xlabel('Number of points in node (or index of point if no parenthesis)')
plt.ylabel(f'Euclidean Distance', fontsize=13)
plt.show()

In [ ]:
# Re-running the Hierarchical clustering based on the correct number of clusters
hclust = AgglomerativeClustering(linkage='ward', affinity='euclidean', n_clusters = 5)
hclust_labels = hclust.fit_predict(perspective_centroids)
perspective_centroids['hclust_labels'] = hclust_labels

In [ ]:
# Mapper between concatenated clusters and hierarchical clusters
cluster_mapper = perspective_centroids['hclust_labels'].to_dict()

#perspective_centroids_2 = perspective_centroids.copy()

# Mapping the hierarchical clusters on the centroids to the observations
km_metric_perspective['merged_labels'] = km_metric_perspective.apply(lambda row: cluster_mapper[(row['Demo_labels'], row['Eng_labels'])], axis=1)

# Merged cluster centroids
km_metric_perspective.groupby('merged_labels').mean()#[metric_features]

## Characterizing Clusters by Perspective

In [ ]:
km_hc_metric_perspective = pva_metric.copy()

km_hc_metric_perspective['Demo_labels'], km_hc_metric_perspective['Eng_labels'], km_hc_metric_perspective['merged_labels'] = km_metric_perspective['Demo_labels'],km_metric_perspective['Eng_labels'], km_metric_perspective['merged_labels'] 

In [ ]:
def cluster_profiles(df, label_columns, figsize, compar_titles=None):
    """
    Pass df with labels columns of one or multiple clustering labels. 
    Then specify this label columns to perform the cluster profile according to them.
    """
    if compar_titles == None:
        compar_titles = [""]*len(label_columns)
        
    sns.set()
    fig, axes = plt.subplots(nrows=len(label_columns), ncols=2, figsize=figsize, squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compar_titles):
        # Filtering df
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df.drop(drop_cols, axis=1)
        
        # Getting the cluster centroids and counts
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]
        
        # Setting Data
        pd.plotting.parallel_coordinates(centroids, label, color=sns.color_palette(), ax=ax[0])
        sns.barplot(x=label, y="counts", data=counts, ax=ax[1])

 

        #Setting Layout
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(len(handles))]
        ax[0].annotate(text=titl, xy=(0.95,1.1), xycoords='axes fraction', fontsize=13, fontweight = 'heavy') 
        ax[0].legend(handles, cluster_labels) # Adaptable to number of clusters
        ax[0].axhline(color="black", linestyle="--")
        ax[0].set_title("Cluster Means - {} Clusters".format(len(handles)), fontsize=13)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=-20)
        ax[1].set_xticklabels(cluster_labels)
        ax[1].set_xlabel("")
        ax[1].set_ylabel("Absolute Frequency")
        ax[1].set_title("Cluster Sizes - {} Clusters".format(len(handles)), fontsize=13)
    
    plt.subplots_adjust(hspace=0.4, top=0.90)
    plt.suptitle("Cluster Simple Profilling", fontsize=23)
    plt.show()

In [ ]:
# Profilling each cluster (product, behavior, merged)
cluster_profiles(
    km_metric_perspective, 
    label_columns = ['Demo_labels', 'Eng_labels', 'merged_labels'],
    figsize = (28, 13),
    compar_titles = ["Demographic Variables Clustering", "Engagement Variables Clustering", "Merged Clustering"]
)

In [ ]:
# Profilling each cluster (product, behavior, merged)
cluster_profiles(
    km_hc_metric_perspective, 
    label_columns = ['Demo_labels', 'Eng_labels', 'merged_labels'],
    figsize = (28, 13),
    compar_titles = ["Demographic Variables Clustering", "Engagement Variables Clustering", "Merged Clustering"]
)

## Cluster Visualization using TSNE

In [ ]:
# This is step can be quite time consuming
two_dim = TSNE(random_state=42).fit_transform(km_metric_perspective)

In [ ]:
# # t-SNE visualization
pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c = km_metric_perspective['Demo_labels'], colormap='viridis', figsize=(15,10))
plt.show()

In [ ]:
# # t-SNE visualization
pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c = km_metric_perspective['Eng_labels'], colormap='viridis', figsize=(15,10))
plt.show()

In [ ]:
pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c = km_metric_perspective['merged_labels'], colormap='viridis', figsize=(15,10))
plt.show()